# Benchmarking Custom Circuits in Tour-de-Gross

This notebook gives a convenient workflow benchmarking custom circuits in PBC form. This relies on the `bicycle_compiler` and `bicycle_numerics` crates.

Before running this notebook, make sure the rust code is compiled by running `cargo build --release` in the root directory.


In [80]:
import asyncio, os, json
import matplotlib.pyplot as plt
from typing import Literal

async def run_command(*cmdlist:list[str], input_data=None):
    """Run the exectuable once for one set of input parameters. Supports streaming data from stdin."""

    process = await asyncio.create_subprocess_exec(
        *[str(x) for x in cmdlist],
        stdin=asyncio.subprocess.PIPE if input_data is not None else None,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )

    stdout, stderr = await process.communicate(
            input=input_data.encode("utf-8") if input_data is not None else None
        )
    if stderr:
        print(f"[stderr]\n{stderr.decode()}")
    return stdout.decode()

def pretty_print_compiled_circuit(string, max_lines=None):
    """Prints a circuit as compiled by bicycle_compiler. Truncates to the first few lines if desired."""

    lines = string.replace("],[","]\n[").split("\n")
    if max_lines is not None:
        lines = lines[:max_lines]
    for line in lines:
        print(line)

## Generate measurement tables

Measurement tables encode optimized circuits for how to address particular Pauli matrices within a code block, see section 3.2 of [Tour de Gross (2506.03094)](https://arxiv.org/abs/2506.03094) and the [Compiler Crate Readme](../crates/bicycle_compiler/README.md).

The gross and two-gross codes have different LPUs and have a different set of Pauli matrices that are easy to synthesize directly. Hence a code must be specified.

In [70]:
code = "gross" # or "two-gross"

In [71]:
measurement_table_path = f"table_{code}.dat"

if not os.path.exists(measurement_table_path):
    print("Generating ", measurement_table_path)
    await run_command("../target/release/bicycle_compiler",
                    f"{code}", "generate",
                    measurement_table_path)
else:
    print(measurement_table_path, "already exists.")

table_gross.dat already exists.


## Compile Custom Circuits

Given a circuit in PBC form, with each rotation or measurement, in JSON list format, `bicycle_compiler` outputs a compiled circuit using the Bicycle ISA instruction set.

See the [Compiler Crate Readme](../crates/bicycle_compiler/README.md) for more information.

In [72]:
async def compile_pbc_circuit(circuit:str, code_:Literal["gross", "two-gross"]):
    # Takes a circuit in PBC form as a json list representation
    # Outputs a Bicycle ISA circuit as a json list

    return await run_command("../target/release/bicycle_compiler",
                                code_,
                                "--measurement-table", f"table_{code_}.dat",
                             input_data=circuit)

In [81]:
test_circuit = """{"Rotation":{"basis":["X","X","I","I","I","I","I","I","I","I","I","Y"],"angle":"0.125"}}
{"Rotation":{"basis":["Z","Z","I","I","I","I","I","I","I","I","I","I"],"angle":"0.5"}}
{"Rotation":{"basis":["X","X","I","I","I","I","I","I","I","I","I","I"],"angle":"-0.125"}}
{"Measurement":{"basis":["Z","X","I","I","I","I","I","I","I","I","I","I"],"flip_result":true}}
{"Measurement":{"basis":["X","I","I","I","I","Z","I","I","I","I","I","I"],"flip_result":false}}
"""

pretty_print_compiled_circuit(await compile_pbc_circuit(test_circuit, code), max_lines=20)

[[[0,{"Measure":{"p1":"X","p7":"I"}}]]
[[0,{"Automorphism":{"x":1,"y":5}}]]
[[0,{"Measure":{"p1":"Y","p7":"I"}}]]
[[0,{"Automorphism":{"x":5,"y":1}}]]
[[0,{"Measure":{"p1":"Y","p7":"I"}}]]
[[1,{"Measure":{"p1":"X","p7":"I"}}]]
[[1,{"Automorphism":{"x":0,"y":5}}]]
[[1,{"Measure":{"p1":"Z","p7":"I"}}]]
[[1,{"Automorphism":{"x":0,"y":1}}]]
[[1,{"Measure":{"p1":"Y","p7":"I"}}]]
[[0,{"Automorphism":{"x":1,"y":5}}]]
[[0,{"Measure":{"p1":"Z","p7":"I"}}]]
[[0,{"Automorphism":{"x":5,"y":1}}]]
[[1,{"Automorphism":{"x":0,"y":1}}]]
[[1,{"Measure":{"p1":"X","p7":"I"}}]]
[[1,{"Automorphism":{"x":0,"y":5}}]]
[[0,{"JointMeasure":{"p1":"X","p7":"I"}}]
[1,{"JointMeasure":{"p1":"Z","p7":"I"}}]]
[[1,{"TGate":{"basis":"Z","primed":false,"adjoint":false}}]]
[[1,{"TGate":{"basis":"X","primed":false,"adjoint":false}}]]


In [82]:
def transverse_field_ising_model(nqubits:int, ntrotter:int, theta_ZZ:float, theta_X:float):
    gates = []

    for _ in range(ntrotter):
        # exp(i theta_ZZ Z_i Z_{i+1}) for all adjacent pairs of qubits
        for i in range(nqubits-1):
            pauli = ["Z" if j in [i,i+1] else "I" for j in range(nqubits)]
            gates.append({"Rotation":{"basis":pauli, "angle":str(theta_ZZ)}})

        # exp(i theta_X X_i) for all qubits
        for i in range(nqubits):
            pauli = ["X" if j == i else "I" for j in range(nqubits)]
            gates.append({"Rotation":{"basis":pauli, "angle":str(theta_X)}})

    return "\n".join(map(json.dumps,gates)).replace(" ","")

In [85]:
pretty_print_compiled_circuit(
    await compile_pbc_circuit(transverse_field_ising_model(20, 5, 0.1, 0.1), code),
    max_lines=20
)

[[[0,{"Measure":{"p1":"Z","p7":"I"}}]]
[[0,{"Automorphism":{"x":5,"y":1}}]]
[[0,{"Measure":{"p1":"Y","p7":"I"}}]]
[[0,{"Automorphism":{"x":1,"y":5}}]]
[[0,{"Measure":{"p1":"Y","p7":"I"}}]]
[[1,{"Measure":{"p1":"Y","p7":"I"}}]]
[[0,{"Automorphism":{"x":5,"y":1}}]]
[[0,{"Measure":{"p1":"X","p7":"I"}}]]
[[0,{"Automorphism":{"x":1,"y":5}}]]
[[0,{"JointMeasure":{"p1":"Z","p7":"I"}}]
[1,{"JointMeasure":{"p1":"Z","p7":"I"}}]]
[[1,{"TGate":{"basis":"Z","primed":false,"adjoint":true}}]]
[[1,{"TGate":{"basis":"X","primed":false,"adjoint":true}}]]
[[1,{"TGate":{"basis":"Z","primed":false,"adjoint":true}}]]
[[1,{"TGate":{"basis":"X","primed":false,"adjoint":false}}]]
[[1,{"TGate":{"basis":"Z","primed":false,"adjoint":false}}]]
[[1,{"TGate":{"basis":"X","primed":false,"adjoint":false}}]]
[[1,{"TGate":{"basis":"Z","primed":false,"adjoint":false}}]]
[[1,{"TGate":{"basis":"X","primed":false,"adjoint":false}}]]
[[1,{"TGate":{"basis":"Z","primed":false,"adjoint":true}}]]


## Circuit Benchmarking


Given a circuit consisting of Bicycle ISA gates, again in JSON list format, `bicycle_numerics` outputs a tally of statistics.

See the [Numerics Crate Readme](../crates/bicycle_numerics/README.md) for more information.

In [86]:
async def benchmark_compiled_circuit(circuit:str, # Bicycle ISA circuit as a json list
                                     qubits:int,
                                     code_:Literal["gross", "two-gross"],
                                     noise:Literal["1e-3", "1e-4"]):
    "Outputs a csv format tallying various statistics for each measurement."

    return await run_command("../target/release/bicycle_numerics",
                            str(qubits), f"{code_}_{noise}",
                             input_data=circuit)

In [87]:
compiled_circuit = await compile_pbc_circuit(test_circuit, code)
print(await benchmark_compiled_circuit(compiled_circuit, 12, code, "1e-4"))

code,p,i,qubits,idles,t_injs,automorphisms,measurements,joint_measurements,measurement_depth,end_time,total_error
gross,0.0001,1,22,0,86,18,16,1,9,17822,0.0000756582614568
gross,0.0001,2,22,2003,86,12,9,1,18,34660,0.00015130945506952
gross,0.0001,3,22,1967,86,12,10,1,27,51618,0.0002269616586304
gross,0.0001,4,22,0,0,12,14,0,40,51738,0.0002269757998444
gross,0.0001,5,22,0,0,18,13,0,53,51738,0.0002269889310584



In [88]:
compiled_circuit = await compile_pbc_circuit(transverse_field_ising_model(20, 5, 0.1, 0.1), code)
print(await benchmark_compiled_circuit(compiled_circuit, 20, code, "1e-4"))

code,p,i,qubits,idles,t_injs,automorphisms,measurements,joint_measurements,measurement_depth,end_time,total_error
gross,0.0001,1,22,57,90,12,10,1,9,18186,0.00007916820081048
gross,0.0001,2,22,2013,90,16,15,1,23,35916,0.0001583414549232
gross,0.0001,3,22,2079,90,4,4,1,26,53646,0.00023750359815976
gross,0.0001,4,22,2064,90,16,15,1,40,71376,0.00031667685234592
gross,0.0001,5,22,1932,90,20,22,1,61,89106,0.0003958571765848
gross,0.0001,6,22,1860,90,28,22,1,82,106836,0.0004750375009628
gross,0.0001,7,22,1962,90,12,10,1,91,124566,0.00055420570451648
gross,0.0001,8,22,1962,90,26,22,1,112,142296,0.00063338602904136
gross,0.0001,9,22,1866,90,20,22,1,133,160026,0.0007125663531852
gross,0.0001,10,22,1860,90,22,22,1,154,177756,0.0007917466775632
gross,0.0001,11,22,1881,90,28,23,1,175,195654,0.00087092801221424
gross,0.0001,12,22,0,90,12,8,0,176,214008,0.00095004609294264
gross,0.0001,13,22,0,90,16,14,0,187,233298,0.00102917023415664
gross,0.0001,14,22,0,90,4,2,0,189,250956,0.00110828225439944
gross